In [ ]:
# Instalando o SDK do Google Gemini
%pip -q install google-genai

# Configura a API Key do Google Gemini
import os
from google.colab import userdata

# Define a chave da API usando dados do usuário do Google Colab
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


##  Bloco 2: Importação de Módulos e Configuração do Cliente


# Importando o cliente da SDK do Google Gemini
from google import genai

# Inicializando o cliente da API do Google Gemini
client = genai.Client()

# Importações adicionais para funcionalidade dos agentes
%pip install google-adk  # Instalação do Framework ADK
from IPython import get_ipython
from IPython.display import display

from google.adk.agents import Agent  # Para criar agentes
from google.adk.runners import Runner  # Para executar os agentes
from google.adk.sessions import InMemorySessionService  # Para gerenciamento de sessões
from google.adk.tools import google_search  # Ferramenta de busca do Google
from google.genai import types  # Para criar conteúdos (Content e Part)

##Bloco 3: Listagem de Modelos Disponíveis
# Cria um serviço de sessão em memória
session_service = InMemorySessionService()

# Lista todos os modelos disponíveis atualmente na API do Gemini
for model in client.models.list():
    print(model.name)

# Definindo os IDs dos modelos que serão utilizados
MODELO_RAPIDO = "gemini-2.0-flash"
MODELO_ROBUSTO = "gemini-1.5-flash"
MODEL_ID = "gemini-2.0-flash"

## Bloco 4: Teste Inicial do Modelo Gemini

# Objetivo da célula: testar o modelo Gemini diretamente
from google.generativeai import GenerativeModel
from IPython.display import HTML, Markdown

# Pergunte ao modelo quando é a próxima imersão de IA
model = GenerativeModel("gemini-1.5-flash")
resposta = model.generate_content("Quando é a próxima Imersão IA com Google Gemini da Alura?")
display(Markdown(f"**Resposta:**\n\n{resposta.text}"))

# Teste da busca com contexto da Google API
response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta obtida
display(Markdown(f"Resposta:\n {response.text}"))
# Exibe a busca realizada e as URLs utilizadas
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))



## Bloco 5: Configuração de Agentes e Funções Auxiliares

# Instalando Framework ADK de agentes do Google
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap  # Para formatar melhor a saída de texto
from IPython.display import display, Markdown  # Para exibir texto formatado no Colab
import requests  # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")  # Ignora avisos de advertência

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
import asyncio  # Importar para usar asyncio.sleep
from google.api_core import exceptions  # Importar para capturar exceções da API

async def call_agent(agent: Agent, message_text: str) -> str:
    max_retries = 5  # Número máximo de tentativas em caso de erro
    initial_delay = 1  # Atraso inicial em segundos

    for attempt in range(max_retries):
        try:
            session_service = InMemorySessionService()  # Cria um novo serviço de sessão
            session = await session_service.create_session(app_name=agent.name, user_id="user1")  # Cria uma nova sessão
            runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)  # Cria um Runner para o agente
            content = types.Content(role="user", parts=[types.Part(text=message_text)])  # Cria o conteúdo da mensagem de entrada

            final_response = ""
            # Itera assincronamente pelos eventos retornados durante a execução do agente
            async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=content):
                if event.is_final_response():
                    for part in event.content.parts:
                        if part.text is not None:
                            final_response += part.text
                            final_response += "\n"
            return final_response  # Retorna a resposta final

        except exceptions.ServerError as e:
            if e.code == 503:  # Verifica se é um erro 503 UNAVAILABLE
                print(f"Erro 503 UNAVAILABLE na tentativa {attempt + 1}/{max_retries}. Tentando novamente...")
                delay = initial_delay * (2 ** attempt)  # Calcula o atraso com backoff exponencial
                await asyncio.sleep(delay)
            else:
                raise e  # Relança exceções não tratadas
        except Exception as e:
            print(f"Erro inesperado na tentativa {attempt + 1}/{max_retries}: {e}")
            raise e

    raise Exception(f"Falha após {max_retries} tentativas de chamar o agente {agent.name}.")  # Levanta exceção final

## Bloco 6: Definição dos Agentes

##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
async def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model=MODELO_RAPIDO,
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]  # Ferramenta de busca do Google
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    # Executa o agente e obtém os lançamentos
    lancamentos = await call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

################################################
# --- Agente 2: Planejador de Posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model=MODELO_ROBUSTO,
        instruction="""
        Você é um planejador de posts que organiza ideias em um formato coerente e criativo.
        Utilizando as notícias mais relevantes, elabore um plano de postagem para redes sociais.
        """,
        description="Agente que planeja posts",
        tools=[google_search]  # Utiliza a ferramenta de busca do Google
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente e obtém o plano de post
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model=MODELO_RAPIDO,
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente e obtém a revisão
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################

def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model=MODELO_RAPIDO,
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente e obtém a revisão
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

## Bloco 7: Execução do Sistema de Criação de Posts

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

from datetime import date  # Importa a função para obter a data atual

# Obtendo a data de hoje
data_de_hoje = date.today().strftime("%d/%m/%Y")

# Início do sistema de criação de posts
print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀\n")

# Solicitar o tópico ao usuário
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Lógica do sistema de agentes
if not topico:
    print("\nVocê esqueceu de digitar o tópico!")
else:
    print(f"\nMaravilha! Vamos então criar o post sobre novidades em {topico}")

    # Chama o agente buscador e aguarda seu resultado
    lancamentos_buscados = await agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    # Exibe o resultado formatado
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    # Chama o agente planejador e aguarda seu resultado
    plano_de_post = await agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    # Exibe o resultado formatado
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    # Chama o agente redator e aguarda seu resultado
    post_redigido = await agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    # Exibe o resultado formatado
    display(to_markdown(post_redigido))
    print("--------------------------------------------------------------")

    # Chama o agente revisor e aguarda seu resultado
    post_final = await agente_revisor(topico, post_redigido)
    print("\n--- 📑 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")


###     BLOCO  8: Função Principal para Coordenação do Ciclo de Agentes

# Função para orquestrar a execução do sistema de criação de posts

async def main():
    print("🚀 Iniciando o Systema de Criação de Posts 🚀\n")
    topico = input("Digite o TÓPICO sobre o qual você deseja criar um post: ")

    if not topico:
        print("Você não digitou um tópico.")
        return

    # Executa os agentes na sequência
    lancamentos_buscados = await agente_buscador(topico, data_de_hoje)
    plano_de_post = await agente_planejador(topico, lancamentos_buscados)
    post_redigido = await agente_redator(topico, plano_de_post)
    post_final = await agente_revisor(topico, post_redigido)

    # Exibe o post final revisado
    print("\n--- Post Final ---\n")
    display(to_markdown(post_final))

# Chamando a função principal
await main()

##    Bloco 9: Adicionando Feedback e Melhoria Contínua (se aplicável)

# Função para coletar feedback sobre o post gerado
def coletar_feedback():
    feedback = input("Você gostaria de fornecer feedback sobre o post gerado? (sim/não): ")
    if feedback.lower() == 'sim':
        comentario = input("Por favor, forneça seu feedback: ")
        print(f"Obrigado pelo feedback: {comentario}")
    else:
        print("Obrigado! Espero que tenha gostado do post.")

## Considerações Finais
##   Estrutura Completa: Agora, o código está organizado com todos os blocos necessários para compreender cada parte do funcionalidade dos agentes.

### Adaptabilidade: Os blocos de código podem ser facilmente adaptados ou expandidos para incluir mais funcionalidades ou otimizações conforme necessário.

##### Feedback: A inclusão de uma seção para coletar feedback possibilita melhorias e ajustes contínuos no sistema, o que é fundamental em projetos de aprendizado de máquina e IA.

###### Testes e Validações: Testar cada bloco de forma independente e dentro do contexto geral é sempre uma boa prática.